In [4]:
import torch
import torch.nn as neunet
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as npy
import torch.nn.functional as Func

transform_train = transforms.Compose([
 transforms.RandomCrop(32, padding=4),
 transforms.RandomHorizontalFlip(),
 transforms.ToTensor(),
 transforms.Normalize((0.5,), (0.5,))

])

transform_test = transforms.Compose([
 transforms.ToTensor(),
 transforms.Normalize((0.5,), (0.5,))
])

batchSize = 128
trainingSetval = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainingLoader = torch.utils.data.DataLoader(trainingSetval, batch_size=batchSize, shuffle=True)
testingSetval = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testingLoaderVal = torch.utils.data.DataLoader(testingSetval, batch_size=batchSize, shuffle=False)

class CNNimprovised(neunet.Module):

 def __init__(self):

  super(CNNimprovised, self).__init__()

  self.convLayer1 = neunet.Conv2d(3, 64, kernel_size=3, padding=1)
  self.convLayer2 = neunet.Conv2d(64, 128, kernel_size=3, padding=1)
  self.convLayer3 = neunet.Conv2d(128, 256, kernel_size=3, padding=1)

  self.batchNormal1 = neunet.BatchNorm2d(64)
  self.batchNormal2 = neunet.BatchNorm2d(128)
  self.batchNormal3 = neunet.BatchNorm2d(256)

  self.pool = neunet.MaxPool2d(2, 2)
  self.dropout = neunet.Dropout(0.4)
  self.fconvLayer1 = neunet.Linear(256 * 4 * 4, 512)
  self.fconvLayer2 = neunet.Linear(512, 10)

 def forward(self, ip):

  ip = self.pool(Func.relu(self.batchNormal1(self.convLayer1(ip))))
  ip = self.pool(Func.relu(self.batchNormal2(self.convLayer2(ip))))
  ip = self.pool(Func.relu(self.batchNormal3(self.convLayer3(ip))))

  ip = ip.view(-1, 256 * 4 * 4)
  ip = self.dropout(Func.relu(self.fconvLayer1(ip)))
  ip = self.fconvLayer2(ip)
  return ip

deviceCpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelVar = CNNimprovised().to(deviceCpu)
optimizerResult = optim.SGD(modelVar.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
schedulerResult = optim.lr_scheduler.StepLR(optimizerResult, step_size=20, gamma=0.1)
criterionResult = neunet.CrossEntropyLoss()

epochCount = 50
for indivEpoch in range(epochCount):
 modelVar.train()
 runningLoss = 0.0
 correct = 0
 total = 0

 for inputVar, labell in trainingLoader:

  inputVar, labell = inputVar.to(deviceCpu), labell.to(deviceCpu)
  optimizerResult.zero_grad()
  outputs = modelVar(inputVar)
  lossValue = criterionResult(outputs, labell)
  lossValue.backward()
  optimizerResult.step()
  runningLoss += lossValue.item()
  _, predictedVal = outputs.max(1)
  total += labell.size(0)
  correct += predictedVal.eq(labell).sum().item()
 schedulerResult.step()
 train_acc = 100 * correct / total

 print(f"Epoch value {indivEpoch+1}: Loss obtained = {runningLoss/len(trainingLoader):.4f}, Train Accuracy obtained = {train_acc:.2f}%")
print("Training Complete!")

modelVar.eval()

correct = 0

total = 0

with torch.no_grad():

 for inputVar, labell in testingLoaderVal:

  inputVar, labell = inputVar.to(deviceCpu), labell.to(deviceCpu)

  outputs = modelVar(inputVar)

  _, predictedVal = outputs.max(1)

  total += labell.size(0)

  correct += predictedVal.eq(labell).sum().item()

testAccuracyValue = 100 * correct / total

print(f"Test Accuracy value: {testAccuracyValue:.2f}%")


resnetEnabled = True

if resnetEnabled:
 print(f"Resnet enabled now")
 modelVar = models.resnet18(pretrained=True)

 modelVar.fc = neunet.Linear(512, 10) # Adjust for CIFAR-10

 modelVar = modelVar.to(deviceCpu)

 optimizerResult = optim.Adam(modelVar.parameters(), lr=0.001)

 criterionResult = neunet.CrossEntropyLoss() # Corrected to 'neunet.CrossEntropyLoss()'

 for indivEpoch in range(20): # Fewer epochs needed for fine-tuning

  modelVar.train()

  runningLoss = 0.0

  correct = 0

  total = 0

  for inputVar, labell in trainingLoader:

   inputVar, labell = inputVar.to(deviceCpu), labell.to(deviceCpu)

   optimizerResult.zero_grad()

   outputs = modelVar(inputVar)

   lossValue = criterionResult(outputs, labell)

   lossValue.backward()

   optimizerResult.step()

   runningLoss += lossValue.item()

   _, predictedVal = outputs.max(1)

   total += labell.size(0)

   correct += predictedVal.eq(labell).sum().item()
  train_acc = 100 * correct / total
  print(f"ResNet Epoch {indivEpoch+1}: Loss = {runningLoss/len(trainingLoader):.4f}, Train Acc = {train_acc:.2f}%")
 print("Resnettuning done!")

Epoch value 1: Loss obtained = 1.5257, Train Accuracy obtained = 43.55%
Epoch value 2: Loss obtained = 1.2094, Train Accuracy obtained = 56.54%
Epoch value 3: Loss obtained = 1.0788, Train Accuracy obtained = 61.34%
Epoch value 4: Loss obtained = 0.9809, Train Accuracy obtained = 65.26%
Epoch value 5: Loss obtained = 0.9094, Train Accuracy obtained = 67.84%
Epoch value 6: Loss obtained = 0.8606, Train Accuracy obtained = 69.67%
Epoch value 7: Loss obtained = 0.8119, Train Accuracy obtained = 71.52%
Epoch value 8: Loss obtained = 0.7775, Train Accuracy obtained = 72.58%
Epoch value 9: Loss obtained = 0.7450, Train Accuracy obtained = 73.79%
Epoch value 10: Loss obtained = 0.7059, Train Accuracy obtained = 75.32%
Epoch value 11: Loss obtained = 0.6885, Train Accuracy obtained = 75.88%
Epoch value 12: Loss obtained = 0.6601, Train Accuracy obtained = 76.93%
Epoch value 13: Loss obtained = 0.6363, Train Accuracy obtained = 77.68%
Epoch value 14: Loss obtained = 0.6123, Train Accuracy obtai

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


Test Accuracy value: 87.21%
Resnet enabled now


100%|██████████| 44.7M/44.7M [00:00<00:00, 210MB/s]


ResNet Epoch 1: Loss = 1.0216, Train Acc = 65.02%
ResNet Epoch 2: Loss = 0.7392, Train Acc = 74.70%
ResNet Epoch 3: Loss = 0.6496, Train Acc = 77.69%
ResNet Epoch 4: Loss = 0.5958, Train Acc = 79.64%
ResNet Epoch 5: Loss = 0.5482, Train Acc = 81.34%
ResNet Epoch 6: Loss = 0.5212, Train Acc = 82.13%
ResNet Epoch 7: Loss = 0.4807, Train Acc = 83.50%
ResNet Epoch 8: Loss = 0.4626, Train Acc = 84.12%
ResNet Epoch 9: Loss = 0.4558, Train Acc = 84.38%
ResNet Epoch 10: Loss = 0.4399, Train Acc = 84.82%
ResNet Epoch 11: Loss = 0.4538, Train Acc = 84.26%
ResNet Epoch 12: Loss = 0.4390, Train Acc = 85.01%
ResNet Epoch 13: Loss = 0.3778, Train Acc = 86.96%
ResNet Epoch 14: Loss = 0.3680, Train Acc = 87.14%
ResNet Epoch 15: Loss = 0.3518, Train Acc = 87.84%
ResNet Epoch 16: Loss = 0.3884, Train Acc = 86.68%
ResNet Epoch 17: Loss = 0.4132, Train Acc = 85.81%
ResNet Epoch 18: Loss = 0.3578, Train Acc = 87.61%
ResNet Epoch 19: Loss = 0.3229, Train Acc = 88.72%
ResNet Epoch 20: Loss = 0.3097, Train Ac